# Basic Visualization Notebook

## MSTICpy Config

In [1]:
%%writefile msticpyconfig.yaml
AzureSentinel:
  Workspaces:
    ASIHuntOMSWorkspaceV4:
      TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
      WorkspaceId: 52b1ab41-869e-4138-9e40-2a4457f09bf0
    CyberSecuritySoc:
      TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
      WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d
    Default:
      TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
      WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d

Overwriting msticpyconfig.yaml


## Import MSTICpy

In [2]:
# Core MSTICPy initialization for Notebooks
from msticpy.nbtools import nbinit
nbinit.init_notebook(namespace=globals());

# Load query providers (typically you'll be using just one)
qry_prov = QueryProvider("AzureSentinel")

Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Connect to Workspace

In [3]:
qry_prov.connect(WorkspaceConfig())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
tables = qry_prov.schema_tables

In [5]:
from bokeh.plotting import figure, output_notebook, show
import ipywidgets as widgets

output_notebook()

Loading BokehJS ...

In [6]:
import ipywidgets as widgets
from IPython.display import display
from bokeh.layouts import row

tableDropdown = widgets.Dropdown(
    options=sorted(tables),
    value=sorted(tables)[0],
    description='Table:',
    disabled=False,
)
display(tableDropdown)

Dropdown(description='Table:', options=('AACAudit', 'AACHttpRequest', 'AADDomainServicesAccountLogon', 'AADDom…

In [9]:
nRows = 10000
table = tableDropdown.value
queryString = f"{tableDropdown.value} | take {nRows}"
df = qry_prov.exec_query(queryString)

# Replace empty string with two quotations for readability
df = df.replace('', '\' \'')

featureColDropdown = widgets.Dropdown(
    options=sorted(list(df)),
    value=sorted(list(df))[0],
    description='Feature Column:',
    disabled=False,
)
display(featureColDropdown)

<IPython.core.display.Javascript object>

Dropdown(description='Feature Column:', options=('AADGroupId', 'AADTarget', 'Actor', 'ActorContextId', 'ActorI…

In [ ]:
# Generate counts for provided feature
feature = featureColDropdown.value
distribution = df[feature].value_counts(dropna=False, normalize=False).rename('Frequency')
distributionNorm = df[feature].value_counts(dropna=False, normalize=True).rename('Probability')

# Graph the distribution
graph = figure(x_range=list(distributionNorm.keys()), plot_height=250, title=f"{feature} Distribution", toolbar_location='below', tools="pan,wheel_zoom,box_zoom,reset")
graph.vbar(x=list(distributionNorm.keys()), top=list(distributionNorm), width=0.9)
graph.xgrid.grid_line_color = None
graph.y_range.start = 0
graph.title.align = 'center'
show(row(graph))
display(pd.concat([distribution, distributionNorm], axis=1))